# Quanta가 보내주는 Input Plan 엑셀파일가공하여 DB에 업데이트

In [1]:
from tool import *

## 1. DB를 불러옴

In [2]:
with open('D:/Data/Quanta Input Plan.bin', 'rb') as f:
    input_df = pickle.load(f)

In [3]:
def make_summary(dataframe):
    df = dataframe.copy()
    df['Series'] = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
    return df.pivot_table('QTY', index='Series', columns='LG Week',aggfunc=sum).fillna(0)
    
def make_input_difference(df):
    d1 = df['Created_at'].unique()[-1]
    d2 = df['Created_at'].unique()[-2]
    c1 = (df['Created_at'] == d1)
    c2 = (df['Created_at'] == d2)
    df1 = input_df[c1]
    df2 = input_df[c2]
    df1 = make_summary(df1)
    df2 = make_summary(df2)
    return get_difference_table(df1, df2, '-\d\d-')

## 2. Quanta Input Plan 불러옴

In [4]:
input_df['Created_at'].unique()[-4:] # 최근 4번의 업데이트 일자 조회

array([datetime.date(2023, 7, 11), datetime.date(2023, 7, 12),
       datetime.date(2023, 7, 13), datetime.date(2023, 7, 14)],
      dtype=object)

In [5]:
path = 'D:/Downloads/' + [name for name in os.listdir('D:/Downloads') if 'The Draft' in name][-1]
path

'D:/Downloads/The Draft of LGE input plan 20230717.xlsx'

In [6]:
updated_date = datetime.date(2023,7,17) # 업데이트된 날짜 설정

In [7]:
df = pd.read_excel(path)
cond = (df.iloc[:, 1] == 'LGE P/N')
i_num = df.loc[cond].index[0]
df = pd.read_excel(path, skiprows=i_num+1)
df = df.drop('MODEL', axis = 1)
df = df[df['LGE P/N'].notnull()]
df = df[~df['Quanta P/N'].isnull()]
df.rename(columns={'LGE P/N':'Mapping Model.Suffix'}, inplace=True)
df.columns = df.columns.astype(str)

if df.columns.str.contains('WK').sum() > 0:
    convert_wk = {}
    for item in get_pattern_from_list(df.columns, re.compile('WK\d+/\d+')):
        new_date = datetime.datetime.strptime(str(datetime.date.today().year) + ' ' + item[2:], '%Y %m/%d') + datetime.timedelta(days=1)
        convert_wk[item] = new_date.strftime('%Y-%m-%d')

    df.rename(columns=convert_wk, inplace=True)

day_list = get_pattern_from_list(df.columns, re.compile('\d\d-\d\d'))
df = df[['Mapping Model.Suffix', 'Quanta P/N']+day_list]
df = df.replace('\s+', 0, regex=True).convert_dtypes()
df = df.fillna(0)

df = df.set_index(['Mapping Model.Suffix', 'Quanta P/N'])
df = df.stack().reset_index()

In [8]:
df = df.rename(columns={'level_2':'Input Date', 0:'QTY'})
df['Input Date'] = pd.to_datetime(df['Input Date'])

df['LG Week'] = df['Input Date'].apply(get_weekname)
df = df[df['QTY'] != 0]
df = df.reset_index(drop=True)
df['Created_at'] = updated_date
df.loc[df['QTY'].apply(type) == type(''), 'QTY'] = 0
df['QTY'] = df['QTY'].astype(int)
df['Mapping Model.Suffix'] = df['Mapping Model.Suffix'].str.strip()

In [9]:
df.shape

(152, 6)

In [10]:
input_df.shape

(41098, 6)

In [11]:
make_summary(df)

LG Week,2023-07-03(W27),2023-07-10(W28),2023-07-17(W29),2023-07-24(W30),2023-07-31(W31),2023-08-07(W32),2023-08-14(W33),2023-08-21(W34),2023-08-28(W35)
Series,,,,,,,,,
11T50Q,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0
14T90R,0.0,70.0,0.0,195.0,0.0,45.0,0.0,90.0,121.0
15U40Q,185.0,0.0,0.0,240.0,240.0,429.0,0.0,0.0,0.0
15U40R,101.0,850.0,0.0,50.0,549.0,1700.0,132.0,0.0,0.0
15U50Q,2.0,109.0,0.0,913.0,360.0,1206.0,340.0,159.0,121.0
15U50R,140.0,69.0,745.0,200.0,200.0,1065.0,236.0,265.0,151.0
16T90Q,0.0,10.0,0.0,0.0,30.0,141.0,190.0,0.0,0.0
16T90R,0.0,355.0,0.0,805.0,230.0,1101.0,99.0,83.0,788.0


In [12]:
get_difference_table(make_summary(df), make_summary(input_df[(input_df['Created_at'] == input_df['Created_at'].max())]), '-\d\d-')

,2023-07-03(W27),2023-07-10(W28),2023-07-17(W29),2023-07-24(W30),2023-07-31(W31),2023-08-07(W32),2023-08-14(W33),2023-08-21(W34),2023-08-28(W35)
11T50Q,0,0,0,0,0,0,0,0,0
14T90R,0,0,0,195,-100,-85,-10,0,0
15U40Q,0,0,0,0,0,0,0,0,0
15U40R,0,0,0,0,0,0,0,0,0
15U50Q,0,0,0,-11,-139,112,38,0,0
15U50R,0,0,30,0,0,-30,0,0,0
16T90Q,0,0,0,-10,0,10,0,0,0
16T90R,0,0,0,765,0,-100,-665,0,0


In [13]:
input_df = pd.concat([input_df, df])
input_df.reset_index(drop=True, inplace=True)

In [14]:
input_df.shape

(41250, 6)

## 3. 최종 결과를 DB에 저장

In [15]:
with open('D:/Data/Quanta Input Plan.bin', 'wb') as f:
    pickle.dump(input_df, f)

In [16]:
with open('D:/Data/DB backup/Quanta Input Plan.bin', 'wb') as f:
    pickle.dump(input_df, f)

### OS별 Quanta 생산계획으로 출력

In [18]:
with open('D:/Data/Quanta Input Plan.bin', 'rb') as f:
    pr = pickle.load(f)

day = pr['Created_at'].max() # 최근일자의 생산계획 가져옴
print('조회일자 : {}'.format(day))
pr = pr[pr['Created_at']==day]

pr = pr.pivot_table('QTY', index=['Mapping Model.Suffix', 'Quanta P/N'], columns='LG Week', aggfunc='sum').fillna(0).reset_index()

col = ['Model.Suffix', 'OS TYPE']
# 클립보드로 붙여넣기
pr.merge(get_pdr()[col], left_on='Mapping Model.Suffix', right_on='Model.Suffix').drop(columns='Model.Suffix').groupby(['OS TYPE']).sum().to_clipboard()
pr.merge(get_pdr()[col], left_on='Mapping Model.Suffix', right_on='Model.Suffix').drop(columns='Model.Suffix').groupby(['OS TYPE', 'Mapping Model.Suffix', 'Quanta P/N']).sum()

조회일자 : 2023-01-06


2023-01-02(W01)  \
OS TYPE                          Mapping Model.Suffix Quanta P/N                     
Chrome OS                        11TG50Q-E.AC10KN     1LI70000087                0   
NON-OS                           15UD40Q-G.AX70K      1LGH0000076                0   
                                 15UD50Q-G.AX30K      1LGHA000110              255   
                                 15UD50Q-G.AX50K      1LGHA000105              889   
                                 16TD90Q-G.AX56K      1NLJA000101                0   
...                                                                            ...   
WINDOWS 11 PRO WITH Downgrade FR 15U50Q-S.AS5KL       1LGHA000146                0   
                                 15U50Q-S.AS5LL       1LGHA000138                0   
                                 15U50Q-S.AS7DL       1LGHA000139                0   
                                 15U50Q-S.AS7GL       1LGHA000140                0   
                                 16T90Q-G.AS7WL       1NLJA000119                0   

                                                                   2023-01-09(W02)  \
OS TYPE                          Mapping Model.Suffix Quanta P/N                     
Chrome OS                        11TG50Q-E.AC10KN     1LI70000087                0   
NON-OS                           15UD40Q-G.AX70K      1LGH0000076                0   
                                 15UD50Q-G.AX30K      1LGHA000110                0   
                                 15UD50Q-G.AX50K      1LGHA000105                0   
                                 16TD90Q-G.AX56K      1NLJA000101                0   
...                                                                            ...   
WINDOWS 11 PRO WITH Downgrade FR 15U50Q-S.AS5KL       1LGHA000146                1   
                                 15U50Q-S.AS5LL       1LGHA000138                1   
                                 15U50Q-S.AS7DL       1LGHA000139                1   
                                 15U50Q-S.AS7GL       1LGHA000140                1   
                                 16T90Q-G.AS7WL       1NLJA000119                0   

                                                                   2023-01-16(W03)  \
OS TYPE                          Mapping Model.Suffix Quanta P/N                     
Chrome OS                        11TG50Q-E.AC10KN     1LI70000087                0   
NON-OS                           15UD40Q-G.AX70K      1LGH0000076              224   
                                 15UD50Q-G.AX30K      1LGHA000110              725   
                                 15UD50Q-G.AX50K      1LGHA000105             1000   
                                 16TD90Q-G.AX56K      1NLJA000101                0   
...                                                                            ...   
WINDOWS 11 PRO WITH Downgrade FR 15U50Q-S.AS5KL       1LGHA000146                0   
                                 15U50Q-S.AS5LL       1LGHA000138                0   
                                 15U50Q-S.AS7DL       1LGHA000139                0   
                                 15U50Q-S.AS7GL       1LGHA000140                0   
                                 16T90Q-G.AS7WL       1NLJA000119                0   

                                                                   2023-01-23(W04)  \
OS TYPE                          Mapping Model.Suffix Quanta P/N                     
Chrome OS                        11TG50Q-E.AC10KN     1LI70000087                0   
NON-OS                           15UD40Q-G.AX70K      1LGH0000076                0   
                                 15UD50Q-G.AX30K      1LGHA000110                0   
                                 15UD50Q-G.AX50K      1LGHA000105                0   
                                 16TD90Q-G.AX56K      1NLJA000101                0   
...                                                                            ...   
WINDOWS 11 PRO WITH Do